<h1><span style="color:red">Examples of Spatial Statistics</span></h1>

Currently works with polygon data in SuAVE



## 1. Retrieve survey parameters

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!rm -rf myclone
!git clone --depth 1 "https://github.com/SteveLamont/Suave_on_Colab.git" myclone

%cd /content/myclone/helpers
!git pull

Cloning into 'myclone'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 81 (delta 20), reused 49 (delta 11), pack-reused 0
Unpacking objects: 100% (81/81), 2.90 MiB | 3.18 MiB/s, done.
/content/myclone/helpers
Already up to date.


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
# common imports
!pip install libpysal
!pip install mgwr
!pip install geopandas
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

def printmd(string):
    display(Markdown(string))

absolutePath = "/content/drive/MyDrive/suave/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

# specific imports
import math
import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
import os

import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
import geopandas as gp
import matplotlib as mpl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 92.2 MB/s eta 0:00:00


## 2. Read the survey file and extract numeric variables

In [ ]:

# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)# print(absolutePath + csv_file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")

col = 0
for var in variables_df.varname.values:
    print(str(col) +":  "+ var)
    col = col+1


# create a dictionary of #number variables with abbreviated and full variable names 
var_list = {n[:n.index('#')]:n for n in variables_df.varname.values if '#number' in n}
printmd("<b><span style='color:red'>Numeric variables:</span></b>")

col = 0
for var in var_list:
    print(str(col) +":  "+ var)
    col = col+1

#create a dataframe of only #number variables
nums_df = df[[n for n in variables_df.varname.values if '#number' in n]]


## 3. Create a geodataframe

In [ ]:
# 1. figure out the geometry field
geometry_vars = [col for col in df.columns if 'geometry' in col]
geometry_vars

In [ ]:
# 2. convert this field to shapely geometry
s = gp.GeoSeries.from_wkt(df[geometry_vars[0]])

In [ ]:
# 3. append the geometry series to the dataframe to create a gdf
gdf = gp.GeoDataFrame(df, crs="EPSG:4326", geometry=s)

In [ ]:
# 4. make sure it worked Ok, by plotting some variable

col_to_map = 'Subjective wellbeing (0-10)#number'
gdf.plot(figsize=(15,5), column=col_to_map,legend=True)
print (col_to_map)

## 4. ESDA and Spatial Autocorrelation 


In [ ]:
import esda # from PySAL
%matplotlib inline

In [ ]:
# General info about the dataset
gdf.info()

In [ ]:
with pd.option_context("display.max_columns", None):
    display(gdf.drop(['geometry'],axis=1))


In [ ]:
# Explore missing values

with pd.option_context("display.max_rows", None):
    display(df.isnull().sum().sort_values(ascending=False))

In [ ]:
# Map selected variable

cntry_name = gdf.columns[0]

gdf.explore(column=col_to_map, cmap ="Blues", scheme="FisherJenks", tiles = "Stamen Watercolor", 
                 tooltip={cntry_name,col_to_map}, popup=True,k=5, highlight=True,
                 width="100%", legend_kwds={"caption":col_to_map, "colorbar":False})

In [ ]:
# Create a histogram

gdf[col_to_map].hist()

## 5. Explore weights and compute spatial similarity

### 5.1 Contiguity-based weights


A commonly-used type of weight is a queen contigutiy weight, which reflects adjacency relationships as a binary indicator variable denoting whether or not a polygon shares an edge or a vertex with another polygon. These weights are symmetric, in that when polygon $A$ neighbors polygon $B$, both $w_{AB} = 1$ and $w_{BA} = 1$.

In [ ]:
wq =  ps.weights.Queen.from_dataframe(gdf)

# In addition to "queen" weights, we can also compute "rook" and "bishop" weights.

In [ ]:
# Plot the weights 
from splot.libpysal import plot_spatial_weights
plot_spatial_weights(wq, gdf, figsize=(15,5))

In [ ]:
# you can use this operation to find neighbors of any country

# e.g., neighbors of Afghanistan:
selected_country = 0
wq.neighbors[selected_country]

In [ ]:
for country in wq.neighbors[selected_country]:
    print(gdf[gdf.columns[0]][country])

### 5.2 Distance-based weights

Neighborhood relationships can be also defined by distance, not just by adjacency (contiguity). Indeed, in many cases distance-based neighbors make more sense.

"K-nearest neighbor weights": distance between centroids of polygons is used to compute binary weights (1 if neighbor, 0 if not), By comparison, "Kernel weights" return some numeric function of the distance between polygon centroids.

## 6. Spatial Autocorrelation ##

The concept of *spatial autocorrelation* relates to the combination of two types of similarity: spatial
similarity and attribute similarity. Although there are many different measures
of spatial autocorrelation, they all combine these two types of simmilarity into
a summary measure.

### 6.1 Spatial Similarity ###

In spatial autocorrelation analysis, the spatial weights
are used to formalize the notion of spatial similarity. There
are many ways to define spatial weights, e.g. using queen contiguity as above.

### 6.2 Attribute Similarity ###

The spatial weight between neighborhoods $i$ and $j$ indicates if the two 
are neighbors (i.e., geographically similar). What we also need is a measure of
attribute similarity to pair up with this concept of spatial similarity. The
**spatial lag** is a derived variable that accomplishes this for us. For neighborhood
$i$ the spatial lag is defined as: $$ylag_i = \sum_j w_{i,j} y_j$$

In [ ]:
# for example, for the variable we explored above (col_to_map):

gdfn = gdf.dropna(subset=[col_to_map])
y = gdfn[col_to_map]
wq =  ps.weights.Queen.from_dataframe(gdfn)
wq.transform = 'r'

ylag = ps.weights.lag_spatial(wq, y)

In [ ]:
# Mapping the spatial lag variable, for visual analysis

gdfn.explore(column=ylag, cmap ="Blues", scheme="FisherJenks", tiles = "Stamen Watercolor", 
                 tooltip={cntry_name,col_to_map}, popup=True,k=5, highlight=True,
                 width="100%", legend_kwds={"caption":"Spatial Lag of " +col_to_map, "colorbar":False})

## 7. Formal statistical assesment of spatial autocorrelation

### 7.1 Join counts

One way to formalize a test for spatial autocorrelation in a binary attribute is
to consider the so-called _joins_. A join exists for each neighbor pair of
observations, and the joins are reflected in our binary spatial weights object
`wq`. 

Each unit can take on one of two values "Black" or "White", analogous to the layout of a chessboard.

For a given pair of neighboring locations there are three different types of joins that can
arise:

- Black Black (BB)
- White White (WW)
- Black White (or White Black) (BW)



With polygon data, we can conduct an analysis using a contiguity matrix. For our selected variable, we need to first discretize the variable we're using; to keep things simple, we'll binarize it using the median so that "high" values are areas with values above the median and "low" values are those below

In [ ]:
# compute the median
y.median()

In [ ]:
# create "low" and "high" values.

# those with values above the median
yb = y > y.median()
sum(yb)

In [ ]:
labels = ["0 Low", "1 High"]
yb = [labels[i] for i in 1*yb] 
gdfn['yb'] = yb

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
gdfn.plot(column='yb', cmap='binary', edgecolor='grey', legend=True, ax=ax)

We know that we have [sum(yb)] "black" polygons. Is the number of Black-Black joins equal to what we would expect if Black polygons were randomply assigned on the map?

In [ ]:
# Using joint counts computation in PySAL:

yb = 1 * (y > y.median()) # convert back to binary
wq =  ps.weights.Queen.from_dataframe(gdfn)
wq.transform = 'b'
np.random.seed(12345)
jc = esda.join_counts.Join_Counts(yb, wq)

In [ ]:
jc.bb # Black Black joins

In [ ]:
jc.ww # White White joins

In [ ]:
jc.bw # Black White joins

Is this a departure from what we would expect if the process generating the spatial distribution of the Black polygons were a completely random one? 

To answer this, PySAL uses random spatial permutations of the observed attribute values to generate a realization under the null of complete spatial randomness (CSR). This is repeated a large number of times (999 default) to construct a reference distribution to evaluate the statistical significance of our observed counts.

The average number of BB joins from the synthetic realizations is:

In [ ]:
jc.mean_bb

Compare it with the observed number of bb joins: is it very different? Is so, we may reject the null hypothesis (complete spatial randomness)

In [ ]:
jc.p_sim_bb # This the p-value

### 7.2 Moran's I

This is a test of global spatial autocorrelation for a continues attribute. For join counts, we binarized the variable, now we use the actual values.



In [ ]:
# For this computation, make sure we don't have null values

gdfn.info()

In [ ]:
# recompute y and queen weights
y = gdfn[col_to_map]
wq =  ps.weights.Queen.from_dataframe(gdfn)

In [ ]:
# regularize the weights (so that sum of weights for each country is 1)

wq.transform = 'r'
np.random.seed(12345)



In [ ]:
mi = esda.moran.Moran(y, wq)
mi.I

In [ ]:
from splot.esda import plot_moran
plot_moran(mi, zstandard=True, figsize=(15,5))
plt.show()

# On the left: reference distribution vs the observed statistic

# On the right: plot of y values against its spatial lag. Moran's I is a slope of this line

In [ ]:
mi.p_sim

### 7.3 Local spatial autocorrelation

Computing hot spots, cold spots, spatial outliers. We've done this before.

## 8. Spatial Regression

From PySAL documentation:

The core idea of spatial econometrics is to introduce a formal representation of space into the statistical framework for regression. This can be done in many ways: by including predictors based on space (e.g. distance to relevant features), by splitting the datasets into subsets that map into different geographical regions (e.g. spatial regimes), by exploiting close distance to other observations to borrow information in the estimation (e.g. kriging), or by introducing variables that put in relation their value at a given location with those in nearby locations, to give a few examples. Some of these approaches can be implemented with standard non-spatial techniques, while others require bespoke models that can deal with the issues introduced. 

### 8.1 Baseline (nonspatial) OLS regression

Essentially, the core of a linear regression is to explain a given variable as a linear function of a set of other characteristics we will collectively call $X_i$:

$$
\ln(P_i) = \alpha + \beta X_i + \epsilon_i
$$




In [ ]:
# We will explore our selected variable and explain it based on several additional variables related to health

x = [
'Women in national parliaments (%)#number',
'Science and tech journal articles (items per bn. PPP$ GDP)#number',
'Mobile broadband subscriptions (per 100)#number',
'Under 5 mortality (per 1000 live births)#number',
'Tuberculosis (per 100,000)#number',
'Homicides (per 100,000)#number',
'Prison population (per 100,000)#number',
'Mean years of schooling (years)#number',
'Undernourishment (%)#number'
]

In [ ]:
gdfn.shape

In [ ]:
# remove rows with missing values
gdfn2 = gdfn.dropna(subset=x, how='any')
gdfn2.shape

In [ ]:
# Explore the dataframe: Scatterplots between X and y; plot the data

plt.scatter(x2.values[:,2], y.values)

In [ ]:
gdfn2.plot()

In [ ]:
x2 = gdfn2[x]
x2.shape

There are many ways to compute OLS regression. Here, we'll use the spreg module in PySAL, which implements a standard OLS routine, but is particularly well suited for regressions on spatial data. 

We'll also build a spatial weights matrix that connects every observation to its 8 nearest neighbors - which results in extra diagnostics from the baseline model though not used in OLS)

In [ ]:
from spreg.ols import OLS

# We'll also scale the independent variables

import sklearn.preprocessing

x2s = sklearn.preprocessing.StandardScaler().fit_transform(x2.values)

In [ ]:
y = gdfn2[col_to_map]
wq =  ps.weights.Queen.from_dataframe(gdfn2)
wq.transform = 'R'
wq

In [ ]:
m1 = OLS(y.values[:, None], x2s, w=wq, spat_diag=True,moran=True,name_x=x2.columns.tolist(),name_y=col_to_map)

In [ ]:
print(m1.summary)

### 8.2. Geographically-weighted regression (GWR)

(from https://methods.sagepub.com/dataset/howtoguide/geographically-weighted-regression-berlin-districts-2018-python)

The fundamental idea behind GWR specifically (and local regression generally) is that the structure of a process being studied may not be the same under all conditions. This concept, sometimes called nonstationarity, is a common facet of cutting-edge models in spatial analysis. It is quite common for effects to vary depending on an observation’s context, group, or relative location. GWR provides a structured way to model this variation.

GWR works by creating a dataset that is “local” to each site and running a regression on that site.

GWR allows us to recognize that the “overall” effect may not be useful, and that an effect estimated from a more narrow focus on the area around some site i may be different. To build this different estimate, GWR uses an N × N spatial weighting matrix, W.

In practice, estimating a GWR requires the model to tradeoff between local and global behavior. In order for the model to capture local variability, Wi must filter out a small set of data around site i.

In [ ]:
# Let's add country centroids

gdfn2['X'] = gdfn2.representative_point().x
gdfn2['Y'] = gdfn2.representative_point().y

In [ ]:
# prepare inputs

gdfn2_y = gdfn2[col_to_map].values.reshape((-1,1))
gdfn2_X = gdfn2[x].values
u = gdfn2['X']
v = gdfn2['Y']
g_coords = list(zip(u,v))

g_X = (gdfn2_X - gdfn2_X.mean(axis=0)) / gdfn2_X.std(axis=0)

g_y = gdfn2_y.reshape((-1,1))

g_y = (gdfn2_y - gdfn2_y.mean(axis=0)) / gdfn2_y.std(axis=0)

In [ ]:
#Calibrate GWR model

gwr_selector = Sel_BW(g_coords, g_y, g_X) # selects bandwidth for kernel
gwr_bw = gwr_selector.search(bw_min=2) 

print(gwr_bw) # bandwidth value: a distance or N nearest neighbors
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw).fit()

In [ ]:
gwr_results.params[0:5]

In [ ]:
gwr_results.localR2[0:5]

In [ ]:
gwr_results.summary()

In [ ]:
# Diplay variable names 

i = 1
for v in x:
    print (str(i) +":  "+v)
    i=i+1